In [1]:
# Google Colab Mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import random as rn
import tensorflow as tf
from tensorflow import keras
import joblib
import math

# keras random seed https://junstar92.tistory.com/98

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

from keras import optimizers, initializers, losses

EPOCHS = 200
LEARNING_RATE = 0.0001
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)
INITIALIZER = initializers.HeNormal(seed=RANDOM_SEED)
LOSS = losses.MeanSquaredError()

# Loss가 np일 때

# X,Y Value들 float으로 변경 or Learning Rate 줄여주기.
# gradient 계산시 0에 가까우면 문제 생김.

In [3]:
# https://aistudy9314.tistory.com/34

DATA_PATH = '/content/drive/MyDrive/aispark/social/'

class DataGenerator(keras.utils.Sequence):
    def __init__(self,
                 Data_PATH: str,
                 batch_size: int,
                 shuffle: bool = False) -> None:
        
        self.encoder_x = joblib.load(DATA_PATH+'encoder_x.pkl')
        self.decoder_x = joblib.load(DATA_PATH+'decoder_x.pkl')
        self.y = joblib.load(DATA_PATH+'y.pkl')

        self.batch_size = batch_size
        self.shuffle = shuffle

        self.on_epoch_end()

    def on_epoch_end(self):
        self.index = np.arange(len(self.encoder_x))
        if self.shuffle:
            np.random.shuffle(self.index)


    def __len__(self):
        return math.ceil(len(self.encoder_x) / self.batch_size)

    def __getitem__(self, index):
        batch_index = self.index[index*self.batch_size:(index+1)*self.batch_size]

        encoder_x_batch = self.encoder_x[ [i for i in batch_index] ]

        decoder_x_batch = self.decoder_x[ [i for i in batch_index] ]

        y_batch = self.y[ [i for i in batch_index] ]

        return [encoder_x_batch,decoder_x_batch], y_batch

In [4]:
train_gen = DataGenerator(DATA_PATH, batch_size = 128, shuffle=True)



In [5]:
# Early Stopping

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='loss',         
                   min_delta=0.01,                
                   patience=15,                 
                   verbose=1,
                   restore_best_weights=True)  


# Model

# https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=chunjein&logNo=221589624838
# https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

In [6]:
tf.random.set_seed(RANDOM_SEED)
tf.keras.backend.clear_session()

from keras.models import Model
from keras.layers import Input, LSTM, Dense, TimeDistributed

BATCH_SIZE = None

ENCODER_INPUT_DIM = train_gen.encoder_x.shape[2]
DECODER_INPUT_DIM = train_gen.decoder_x.shape[2]

ENCODER_TIMESTEP = 48
DECODER_TIMESTEP = 72
LATENT_DIM = 256

# rnn 계열은 무조건 tanh 사용해야 gpu학습 가능.

# Encoder

encoder_inputs = Input(shape=(ENCODER_TIMESTEP, ENCODER_INPUT_DIM))

lstm_enc, latent_c_1, latent_h_1 = LSTM(LATENT_DIM, activation='tanh', return_sequences=True, return_state=True)(encoder_inputs)

_, latent_c_2, latent_h_2 = LSTM(LATENT_DIM, activation='tanh', return_sequences=False, return_state=True)(lstm_enc) # return_sequences = True는 timestep별 출력 , return_state = True, last_hidden_state 출력


lstm1_output = [latent_c_1, latent_h_1]
lstm2_output = [latent_c_2, latent_h_2]

# Decoder

decoder_inputs = Input(shape=(DECODER_TIMESTEP, DECODER_INPUT_DIM))

lstm_dec = LSTM(LATENT_DIM, activation='tanh', return_sequences=True)(decoder_inputs, initial_state=lstm1_output)

step, latent_c, latent_h = LSTM(LATENT_DIM, activation='tanh', return_sequences=True, return_state=True)(lstm_dec,initial_state=lstm2_output)


# Regressor

# TimeDistributed
# https://medium.com/smileinnovation/how-to-work-with-time-distributed-data-in-a-neural-network-b8b39aa4ce00


regressor_result = TimeDistributed(layer = Dense(1))(step) 

   
model = Model([encoder_inputs, decoder_inputs], regressor_result)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 28)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 72, 21)]     0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 48, 256)      291840      ['input_1[0][0]']                
                                                                                                  
 lstm_2 (LSTM)                  (None, 72, 256)      284672      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.compile(loss=LOSS, optimizer=OPTIMIZER)
model.fit(train_gen, epochs=EPOCHS, callbacks=[es], shuffle=True)

Epoch 1/200
2321/2321 [==============================] - 101s 39ms/step - loss: 0.0037 - accuracy: 0.0050
Epoch 2/200
2321/2321 [==============================] - 91s 39ms/step - loss: 0.0034 - accuracy: 0.0050
Epoch 3/200
1277/2321 [===============>..............] - ETA: 41s - loss: 0.0033 - accuracy: 0.0050

In [14]:
 # import joblib


model.save('/content/drive/MyDrive/aispark/social/model_f.h5')

['/content/drive/MyDrive/aispark/social/model.pkl']

# 추론

In [15]:

# model_new = keras.models.load_model('/content/drive/MyDrive/aispark/social/model_f.h5')

PATH = '/content/drive/MyDrive/aispark/social/'

test_encoder_x = joblib.load(PATH+'test_encoder_x.pkl') 

test_decoder_x = joblib.load(PATH+'test_decoder_x.pkl') 

In [16]:
y_pred = model.predict([test_encoder_x,test_decoder_x])

34/34 [==============================] - 1s 6ms/step


In [17]:
y_pred.shape # (1088, 72, 1) 1088x72 = 78336

(1088, 72, 1)

In [18]:
y_pred.max()

0.4081587

In [19]:
result = pd.read_csv('/content/drive/MyDrive/aispark/social/answer_sample.csv')
result.shape

(78336, 4)

In [20]:
result['PM2.5'] = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1],1)

In [21]:
result.to_csv('/content/drive/MyDrive/aispark/social/result.csv')